In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malawi-health-data/Data/SampleSubmission.csv
/kaggle/input/malawi-health-data/Data/Train.csv
/kaggle/input/malawi-health-data/Data/Test.csv
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 2.xlsx
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 5.xlsx
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 6.xlsx
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 4.xlsx
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 3.xlsx
/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel/TG Booklet 1.xlsx


In [ ]:

!pip install transformers>=4.32.0 optimum>=1.12.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install langchain
!pip install chromadb
!pip install sentence_transformers # ==2.2.2
!pip install unstructured
!pip install unstructured-pytesseract
!pip install unstructured-inference
!pip install faiss-gpu

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.3 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install -U langchain-huggingface

In [5]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

model_name = "TheBloke/Llama-2-7B-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=60
gen_cfg.temperature=0.01 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.1

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)

2024-07-05 06:39:48.328853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 06:39:48.328970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 06:39:48.426361: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-7B-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
import os
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd

In [8]:
csv_df= pd.read_csv("/kaggle/input/malawi-health-data/Data/Train.csv")

In [9]:
train_qa_df= csv_df[["Question Text","Question Answer"]]

In [10]:
train_qa_df.head()

,Question Text,Question Answer
0,Compare the laboratory confirmation methods fo...,Chikungunya is confirmed using serological tes...
1,When should specimens be collected for Anthrax...,Specimens should be collected during the vesic...
2,Which key information should be recorded durin...,"During a register review, key information abou..."
3,Why is the District log of suspected outbreaks...,The log includes information about response ac...
4,What do Community based surveillance strategie...,Community-based surveillance strategies focus ...


In [11]:
path = "/kaggle/input/malawi-health-data/Data/MWTGBookletsExcel"

In [12]:
books = os.listdir(path)
books

['TG Booklet 2.xlsx',
 'TG Booklet 5.xlsx',
 'TG Booklet 6.xlsx',
 'TG Booklet 4.xlsx',
 'TG Booklet 3.xlsx',
 'TG Booklet 1.xlsx']

In [13]:
loaders = [UnstructuredExcelLoader(f"{path}/{i}") for i in books]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [14]:
test_qa_df=train_qa_df[650:750]

In [15]:
test_qa_df

,Question Text,Question Answer
650,What is an operational case definition?,An operational case definition refers to a set...
651,How can one ensure correct interpretation of o...,Correct interpretation of outbreak data involv...
652,What are the recommendations for specimen coll...,"Asymptomatic contacts, especially in high-inte..."
653,What key activities and measures are outlined ...,Section 6 outlines key activities and measures...
654,"What are the poliovirus serotypes, and how is ...","Poliovirus serotypes are 1, 2, and 3, primaril..."
...,...,...
743,Which section of the guidelines provides a des...,Section 11.0 of these 3rd Edition Malawi IDSR ...
744,"Does MEF stand for an abbreviation in the TG, ...",Medical Teams International
745,In what ways do the verification and documenta...,"In emergency contexts, verification and docume..."
746,What role does the examination of burial cerem...,Examining burial ceremonies helps identify pot...


In [16]:
train_qa_df= train_qa_df[:700]

In [17]:
train_qa_df

,Question Text,Question Answer
0,Compare the laboratory confirmation methods fo...,Chikungunya is confirmed using serological tes...
1,When should specimens be collected for Anthrax...,Specimens should be collected during the vesic...
2,Which key information should be recorded durin...,"During a register review, key information abou..."
3,Why is the District log of suspected outbreaks...,The log includes information about response ac...
4,What do Community based surveillance strategie...,Community-based surveillance strategies focus ...
...,...,...
695,What are some examples of analytical study des...,Analytical study designs such as case-control ...
696,What factors should be considered when interpr...,"When interpreting results, factors to consider..."
697,Why is calculating a case fatality rate import...,Calculating a case fatality rate is important ...
698,"What is the surveillance goal for malaria, and...",The surveillance goal is to detect malaria cas...


In [18]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/kaggle/input/malawi-health-data/Data/Train.csv")

data = loader.load()

# print(data)

In [19]:
docs+=data

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [21]:
 len(all_splits)

3336

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
db_web = FAISS.from_documents(all_splits, embeddings)

In [24]:
db_web.save_local("/kaggle/working/faiss_db")

In [25]:
query = "What is the definition of unusual event"

In [26]:
new_db = FAISS.load_local("/kaggle/working/faiss_db", embeddings, allow_dangerous_deserialization=True)

docs = new_db.similarity_search(query)

In [27]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [28]:
prompt_template='''
 [INST]

<<SYS>>

 Use the following context to Answer the question at the end. Do not use any other information.
 If you can't find the relevant information in the context,
 just say you don't have enough information to answer the question.
 Don't try to make up an answer.
 Give the answer in a single paragraph not more than in 60 words.
 Don't add anything before only provide the answer content.

<</SYS>>

 {context}

User Prompt: {question}

[/INST]
'''

In [29]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_web = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=new_db .as_retriever(search_kwargs={'k': 4, 'score_threshold': 0.8}), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)

In [30]:
query = "Compare the laboratory confirmation methods for Chikungunya and diabetes, and which diseases are diagnosed through blood glucose measurements?"
result = Chain_web(query)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [31]:
final_result = result['result']

# Find the part after [/INST]
output = final_result.split("[/INST]")[1].strip()

print("Result=",output)

Result= Chikungunya and diabetes are diagnosed through different laboratory confirmation methods. Chikungunya is confirmed using serological tests and PCR, while diabetes diagnosis involves blood glucose measurements.


In [32]:
querry_lst=[]
response_lst=[]

In [33]:
# test_qa_df['Generated_Response']

In [34]:
%pip install nltk
# %pip install rouge-score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [35]:
%pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=20e9a9f618cae9efb418b99b2a18efce25e178f703cf44e8596ca0756b08d378
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [36]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from nltk.translate.meteor_score import meteor_score

In [37]:
querry =[]
response=[]

blue=[]
rouge1=[]
rouge2=[]
rougeL=[]
time_data=[]

In [38]:
time_data=[]

In [39]:
%%time
qes=test_qa_df["Question Text"][735]
ans=test_qa_df["Question Answer"][735]


CPU times: user 628 µs, sys: 0 ns, total: 628 µs
Wall time: 633 µs


In [40]:
def calculate_rouge(predicted, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, predicted)
    return scores

In [41]:
def calculate_bleu(predicted, reference):
    reference = [reference.split()]
    predicted = predicted.split()
    smooth = SmoothingFunction().method4
    score = sentence_bleu(reference, predicted, smoothing_function=smooth)
    return score

In [42]:
def calculate_meteor(predicted, reference):
    score = meteor_score([reference], predicted)
    return score


In [43]:
from time import time

In [44]:
# %%time
# qes=train_qa_df["Question Text"][625]
# ans=train_qa_df["Question Answer"][625]

In [45]:
print(qes)

Compare the goals of surveillance for Anthrax and bacterial meningitis.


In [46]:
querry =[]
response=[]

blue=[]
rouge1=[]
rouge2=[]
rougeL=[]
time_data=[]

In [47]:
for i in range(680,700):
    qes=test_qa_df["Question Text"][i]
    ans=test_qa_df["Question Answer"][i]
    time1= time()
    result = Chain_web(qes)
    final_result = result['result']
    output = final_result.split("[/INST]")[1].strip()
    
    time2=time()
    final_time=abs(time2-time1)
    rouge_scores = calculate_rouge(output, ans)
    bleu_score = calculate_bleu(output, ans)
    querry.append(qes)
    response.append(output)
    blue.append(bleu_score)
    rouge1.append(rouge_scores['rouge1'])
    rouge2.append(rouge_scores['rouge2'])
    rougeL.append(rouge_scores['rougeL'])
    time_data.append(final_time)

    

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [48]:
# time1= time()
# result = Chain_web(qes)
# final_result = result['result']
# output = final_result.split("[/INST]")[1].strip()
# querry.append(qes)
# response.append(output)
# time2=time()

# final_time=abs(time2-time1)


In [49]:
print(qes)
print('\n')
print(output)



What actions should be taken at the alert threshold for a foodborne illness?


At the alert threshold for a foodborne illness, immediate actions to take include reporting the illness to the next level of the health system, collecting specimens for laboratory confirmation from patients and suspected food items, and treating suspected cases.


In [50]:
print(ans)

At the alert threshold, if it is observed that two or more people are ill and have consumed food from a common source, immediate actions include reporting the illness to the next level of the health system, collecting specimens for laboratory confirmation from patients and suspected food items, and treating suspected cases.


In [51]:
# output="The intervals that should be monitored for detection of immediately reportable diseases are: The interval between the onset of the first known case and when the case was seen in the health facilityReporting from the community to the health facility and to the district (within 24 hours of onset of illness)"

In [52]:
# output_2 = output.split(':')[1].strip()

# print(output_2)

In [53]:
# rouge_scores = calculate_rouge(output, ans)
# bleu_score = calculate_bleu(output, ans)
# querry.append(qes)
# response.append(output)
# blue.append(bleu_score)
# rouge1.append(rouge_scores['rouge1'])
# rouge2.append(rouge_scores['rouge2'])
# rougeL.append(rouge_scores['rougeL'])
# time_data.append(final_time)




In [54]:
# print(querry)
print(response)

['According to the reference document (TG Booklet 6), RTA stands for Road Traffic Accident. You can find this information on page 106 of the document.', 'Alert verification should start immediately once the alert is received by sub-national and national surveillance focal points, as stated in TG Booklet 5, Paragraph 247.', "Parallel reporting refers to the simultaneous use of multiple reporting mechanisms for the same information, which can lead to confusion and complications during an outbreak. It is important to avoid parallel reporting to maintain clarity, streamline communication, and ensure effective monitoring of the outbreak's progress.", 'To intensify surveillance with neighboring districts and prevent the outbreak from spreading, neighboring districts should collaborate on information sharing, joint surveillance, and response activities. The establishment of cross-border disease surveillance and response committees can facilitate sharing epidemi', 'Laboratory-based surveillanc

In [55]:
# print(querry)
# print(response)
print(blue)
print(rouge1)
print(rouge2)
print(rougeL)
print(time_data)


[0.11980587625285534, 0.6158787442830632, 0.49110727022568434, 0.6247647055287531, 0.3436789057016855, 0.35611184058494927, 0.19259074009082888, 0.13634738360240478, 0.7221052041889016, 0.1834283688193615, 0.4083503543664229, 0.0681597458770418, 0.45486727739500965, 0.6273978076133727, 0, 0.40856390592219133, 0.4454244787882928, 0.22651764613604333, 0.6490953126499728, 0.5313995685244156]
[Score(precision=0.12, recall=1.0, fmeasure=0.21428571428571425), Score(precision=0.6923076923076923, recall=1.0, fmeasure=0.8181818181818181), Score(precision=0.6956521739130435, recall=0.9411764705882353, fmeasure=0.7999999999999999), Score(precision=0.7894736842105263, recall=0.8108108108108109, fmeasure=0.8), Score(precision=0.5, recall=0.6060606060606061, fmeasure=0.5479452054794521), Score(precision=0.43478260869565216, recall=1.0, fmeasure=0.6060606060606061), Score(precision=0.25, recall=1.0, fmeasure=0.4), Score(precision=0.4146341463414634, recall=0.4358974358974359, fmeasure=0.4250000000000

In [56]:
sum(time_data)/len(time_data)

39.6310706615448

In [57]:
# querry.pop()
# response.pop()
# blue.pop()
# rouge1.pop()
# rouge2.pop()
# rougeL.pop()
# time_data.pop()

In [58]:
# print("ROUGE-1: ", rouge_scores['rouge1'])
# print("ROUGE-2: ", rouge_scores['rouge2'])
# print("ROUGE-L: ", rouge_scores['rougeL'])
# print("BLEU: ", bleu_score)


In [59]:
# from rouge_score import rouge_scorer
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [60]:
# import pandas as pd

In [61]:
# test_qa_df.to_csv("/kaggle/working/qa_2_file.csv")

In [62]:
# print(blue)

In [63]:
len_bleu= len(blue)
# len_bleu

In [64]:
sum_bleu= sum(blue)
sum_bleu

7.6055951365512495

In [69]:
sum_bleu/20

0.3802797568275625

In [66]:
len(rouge1)

20

In [67]:
precision_sum1= 0
for i in range(20):
    precision_sum1+=rouge1[i][0]
print('precision_sum1',precision_sum1/20)    

precision_sum1 0.5578074902619732


In [68]:
precision_sum/20

NameError: name 'precision_sum' is not defined

In [70]:
recall_sum1= 0
for i in range(20):
    recall_sum1+=rouge1[i][1]
print("recall_sum1",recall_sum1/20)

recall_sum1 0.7702078932476305


In [ ]:
recall_sum/20

In [71]:
fmeasure_sum1= 0
for i in range(20):
    fmeasure_sum1+=rouge1[i][2]
print("fmeasue_sum1",fmeasure_sum1/20)

fmeasue_sum1 0.599599673285469


In [ ]:
fmeasure_sum1/20

In [72]:
precision_sum2= 0
recall_sum2=0
fmeasure_sum2=0
for i in range(20):
    precision_sum2+=rouge2[i][0]
    recall_sum2+=rouge2[i][1]
    fmeasure_sum2+=rouge2[i][2]
print("precision_sum2",precision_sum2/20)
print("recall_sum2",recall_sum2/20)
print("fmeasure_sum2",fmeasure_sum2/20)

precision_sum2 0.4710543217276963
recall_sum2 0.680643997900685
fmeasure_sum2 0.5067172003596021


In [73]:
precision_sum_l= 0
recall_sum_l=0
fmeasure_sum_l=0
for i in range(20):
    precision_sum_l+=rougeL[i][0]
    recall_sum_l+=rougeL[i][1]
    fmeasure_sum_l+=rougeL[i][2]
print("precision_sumL",precision_sum_l/20)
print("recall_sumL",recall_sum_l/20)
print("fmeasure_sumL",fmeasure_sum_l/20)

precision_sumL 0.5114778924634369
recall_sumL 0.7202643153159424
fmeasure_sumL 0.551850679836521
